In [1]:
import numpy as np

In [2]:
np.random.seed(20520864)

In [3]:
def initialize_population( num_individuals, num_variables ):
    """
    Khởi tạo quần thể gồm num_individuals cá thể. Mỗi cá thể có num_variables biến.
    
    Arguments:
    num_individuals -- Số lượng cá thể
    num_variables -- Số lượng biến
    
    Returns:
    pop -- Ma trận (num_individuals, num_variables ) chứa quần thể mới được khởi tạo ngẫu nhiên.
    """
    assert num_individuals >= 2
    zeros = np.zeros((num_individuals//2, num_variables), dtype=np.int32)
    ones = np.ones((num_individuals - num_individuals//2, num_variables), dtype=np.int32)
    pop = np.vstack((zeros,ones))
    for i in range(num_variables):
        np.random.shuffle(pop[:,i])

    return pop

In [4]:
pop = initialize_population(8,4)
print(pop)

[[1 0 1 0]
 [1 0 1 1]
 [1 1 1 0]
 [0 1 0 1]
 [1 1 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 1 1]]


In [5]:
def onemax( ind ):
    """
    Hàm đánh giá OneMax: Đếm số bit 1 trong chuỗi nhị phân (cá thể ind).
    
    Arguments:
    ind -- Cá thể cần được đánh giá.

    Returns:
    value -- Giá trị của cá thể ind.
    """
    
    return np.sum(ind)

In [6]:
pop_fitness = np.array([onemax(pop[i,:]) for i in range(pop.shape[0])])
print(pop)
print(pop_fitness)

[[1 0 1 0]
 [1 0 1 1]
 [1 1 1 0]
 [0 1 0 1]
 [1 1 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 1 1]]
[2 3 3 2 2 2 0 2]


In [7]:
def better_variation(pop, pool_over_pop_size):
    def count_one_gene(col):
        return np.sum(col)

    num_individuals = pop.shape[0]
    num_variables = pop.shape[1]

    offspring = pop.copy()
    for i in range(pool_over_pop_size - 2):
        offspring = np.vstack((offspring, pop.copy()))
    for i in range(num_variables):
        np.random.shuffle(offspring[:,i])

    dominant_gene = np.zeros(num_variables, dtype=np.int32)
    for i in range(num_variables):
        if count_one_gene(pop[:,i]) >= num_individuals / 2.0:
            dominant_gene[i] = 1
    offspring[0] = dominant_gene
    
    return offspring

In [8]:
print(len(better_variation(pop,2)))
print(len(better_variation(pop,3)))

8
16


In [9]:
print(pop)
better_variation(pop, 2)

[[1 0 1 0]
 [1 0 1 1]
 [1 1 1 0]
 [0 1 0 1]
 [1 1 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 1 1]]


array([[1, 1, 1, 1],
       [1, 0, 1, 0],
       [1, 1, 1, 0],
       [1, 0, 1, 0],
       [1, 1, 0, 1],
       [0, 1, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 0]], dtype=int32)

In [10]:
def uniform_crossover(pop, pool_over_pop_size):
    num_individuals = pop.shape[0]
    num_variables = pop.shape[1]

    offspring = pop.copy()
    for i in range(pool_over_pop_size - 2):
        offspring = np.vstack((offspring, pop.copy()))
    np.random.shuffle(offspring)

    for ind in range(0, num_individuals - 1, 2):
        for gene in range(0, num_variables):
            r = np.random.rand()
            if r < 0.5:
                temp = offspring[ind+1][gene] 
                offspring[ind+1][gene] = offspring[ind][gene]
                offspring[ind][gene] = temp

    return offspring

In [11]:
print(len(uniform_crossover(pop,2)))
print(len(uniform_crossover(pop,3)))

8
16


In [12]:
print(pop)
uniform_crossover(pop, 2)

[[1 0 1 0]
 [1 0 1 1]
 [1 1 1 0]
 [0 1 0 1]
 [1 1 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 1 1]]


array([[1, 0, 1, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 1],
       [0, 1, 0, 1],
       [1, 1, 1, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 0]], dtype=int32)

In [13]:
class POPOP():
    def __init__(self, initialization, variation, evaluation, pool_over_pop_size=2, selection_pressure=2):
        self.initialization = initialization
        self.variation = variation
        self.evaluation = evaluation
        self.pool_over_pop_size = pool_over_pop_size
        self.selection_pressure = selection_pressure

    def tournament_selection(self, pool, pool_fitness):
        pool_size = pool.shape[0]
        group_size = self.pool_over_pop_size * self.selection_pressure
        assert pool_size >= group_size and pool_size % group_size == 0

        new_pop = []

        num_individuals = pool.shape[0]

        for i in range(self.selection_pressure):
            shuffled_idx = np.random.permutation(num_individuals)
            shuffled_pool = pool[shuffled_idx]
            shuffled_pool_fitness = pool_fitness[shuffled_idx]

            for j in range(0, num_individuals, group_size):
                group = shuffled_pool[j:j + group_size]
                group_fitness = shuffled_pool_fitness[j:j + group_size]
                winner = group[group_fitness.argmax()]
                new_pop.append(winner)

        new_pop = np.array(new_pop)
        return new_pop

    def __call__(self, num_individuals, num_variables, num_generations):
        pop = self.initialization(num_individuals, num_variables)
        pop_fitness = np.array([self.evaluation(ind) for ind in pop])
        print('#First Population:')
        print(pop)
        print("#Gen 0:")
        print(pop_fitness)

        for i in range(num_generations):
            offspring = self.variation(pop, self.pool_over_pop_size)
            offspring_fitness = np.array([self.evaluation(ind) for ind in offspring])
            pool = np.vstack((pop, offspring))
            pool_fitness = np.hstack((pop_fitness, offspring_fitness))
            pop = self.tournament_selection(pool, pool_fitness)

            pop_fitness = np.array([self.evaluation(ind) for ind in pop])
            print(f"#Gen {i+1}:")
            print(pop_fitness)
            if np.all(pop == pop[0]):
                print("Population converged")
                break

        print('#Best Individual:')
        print(pop[pop_fitness.argmax()])

In [14]:
genetic_algorithm = POPOP(initialization=initialize_population, variation=uniform_crossover, evaluation=onemax)
print(pop)
genetic_algorithm.tournament_selection(pop,pop_fitness)

[[1 0 1 0]
 [1 0 1 1]
 [1 1 1 0]
 [0 1 0 1]
 [1 1 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 1 1]]


array([[1, 1, 1, 0],
       [1, 0, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 1, 0]], dtype=int32)

In [15]:
np.random.seed(20520864)
num_variables = 8
num_individuals = 8
num_generations = 50
genetic_algorithm(num_individuals, num_variables, num_generations)
# Converge at 5th generation

#First Population:
[[1 0 1 0 0 1 1 1]
 [1 0 1 1 0 1 0 1]
 [1 1 1 0 0 0 1 0]
 [0 1 0 1 1 0 0 0]
 [1 1 0 0 1 1 1 1]
 [0 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0]
 [0 0 1 1 1 1 0 1]]
#Gen 0:
[5 5 4 3 6 2 2 5]
#Gen 1:
[5 5 6 6 6 6 5 6]
#Gen 2:
[6 6 7 6 6 6 7 6]
#Gen 3:
[7 6 6 8 7 6 7 8]
#Gen 4:
[8 8 7 8 8 8 8 7]
#Gen 5:
[8 8 8 8 8 8 8 8]
Population converged
#Best Individual:
[1 1 1 1 1 1 1 1]


In [16]:
np.random.seed(20520864)
num_variables = 16
num_individuals = 12
num_generations = 50
genetic_algorithm(num_individuals, num_variables, num_generations)
# Converge at 11th generation

#First Population:
[[0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0]
 [1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 0]
 [1 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0]
 [1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1]
 [0 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0]
 [1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1]
 [1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 1]
 [0 1 0 1 0 1 0 1 0 0 0 0 1 1 1 1]
 [0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1]
 [0 1 1 1 1 0 0 0 1 0 1 0 1 1 0 0]
 [0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1]
 [1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0]]
#Gen 0:
[ 5  9  8  9  6  7 10  8  7  8 12  7]
#Gen 1:
[11  8 10 10 12  9  9 12 10  8  8 10]
#Gen 2:
[10 11 12 11 10 12 12 12 11 11 12 11]
#Gen 3:
[12 12 12 13 12 13 12 12 12 13 12 13]
#Gen 4:
[13 13 14 13 13 13 14 13 12 13 12 13]
#Gen 5:
[14 14 13 14 14 14 13 14 14 13 14 14]
#Gen 6:
[14 14 14 15 14 14 15 15 14 14 14 14]
#Gen 7:
[15 15 15 15 15 14 15 15 15 15 15 15]
#Gen 8:
[15 15 15 16 15 16 15 15 16 15 15 15]
#Gen 9:
[16 16 16 15 16 15 15 16 16 15 16 16]
#Gen 10:
[16 16 16 16 16 16 16 16 16 15 16 16]
#Gen 11:
[16 16 16 16 16 16 16 16 16 16 16 16]
Popula

In [17]:
np.random.seed(20520864)
num_variables = 32
num_individuals = 20
num_generations = 50
genetic_algorithm(num_individuals, num_variables, num_generations)
# Converge at 14th generation

#First Population:
[[0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1]
 [1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 0 0 1 1 1 0 0 1 0 1 1 0 1 0 1 0 1]
 [0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 0]
 [1 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0]
 [1 1 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 1 1 0]
 [0 1 1 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1]
 [1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 1 1 0 0 1 0 0 1 1]
 [0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 0 0 1 1]
 [1 0 1 0 0 0 0 1 0 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0]
 [0 1 1 0 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1]
 [1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 0 0 1]
 [1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 1]
 [1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1

In [18]:
np.random.seed(20520864)
num_variables = 64
num_individuals = 38
num_generations = 50
genetic_algorithm(num_individuals, num_variables, num_generations)
# Converge at 17th generation

#First Population:
[[0 0 0 ... 1 1 1]
 [1 1 0 ... 1 1 0]
 [0 1 0 ... 1 1 1]
 ...
 [1 0 1 ... 1 0 1]
 [0 1 0 ... 1 1 1]
 [1 1 1 ... 0 1 0]]
#Gen 0:
[27 32 26 27 37 25 32 38 28 31 24 33 28 31 32 34 35 34 35 36 31 33 23 38
 37 29 28 34 29 32 39 34 34 36 34 32 38 30]
#Gen 1:
[36 34 34 38 35 34 38 36 39 38 36 35 34 37 39 41 34 34 38 35 31 39 37 35
 38 35 36 34 36 38 39 37 41 34 37 38 35 36]
#Gen 2:
[38 39 42 41 47 38 38 38 43 38 38 37 36 40 40 37 38 39 38 38 40 38 37 39
 38 38 42 38 47 37 43 37 38 37 39 41 39 39]
#Gen 3:
[44 46 42 43 40 43 41 44 43 44 40 47 39 42 47 39 45 38 41 39 45 41 47 42
 38 44 43 39 40 41 39 41 41 42 43 47 44 42]
#Gen 4:
[42 45 44 47 44 47 47 46 44 44 42 43 44 46 46 47 47 47 45 42 45 43 46 47
 43 47 46 46 42 47 47 46 44 44 45 45 47 40]
#Gen 5:
[47 48 46 47 46 50 47 48 46 47 50 46 47 44 47 50 49 47 46 47 47 46 47 47
 47 47 50 50 47 45 47 48 47 47 47 49 45 50]
#Gen 6:
[48 48 48 50 47 49 52 53 54 48 50 53 47 47 50 47 47 47 53 50 51 49 47 47
 50 54 48 47 50 53 53 51 48 50

In [19]:
np.random.seed(20520864)
num_variables = 128
num_individuals = 50
num_generations = 50
genetic_algorithm(num_individuals, num_variables, num_generations)
# Converge at 27th generation

#First Population:
[[0 0 1 ... 0 1 0]
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 1]
 ...
 [1 0 0 ... 1 0 0]
 [0 0 1 ... 1 0 0]
 [1 1 0 ... 1 1 0]]
#Gen 0:
[66 55 61 63 62 69 58 65 63 51 65 53 67 64 56 71 56 59 66 59 60 70 59 59
 62 60 63 71 69 73 70 65 57 64 71 70 69 66 65 68 51 63 68 60 71 66 70 69
 72 70]
#Gen 1:
[67 67 66 71 70 70 65 71 73 66 65 72 70 69 71 69 64 70 70 69 72 71 69 71
 72 65 73 72 70 71 70 67 71 71 69 66 70 66 72 69 70 67 71 70 66 72 71 71
 69 66]
#Gen 2:
[72 76 77 71 72 72 73 77 72 69 74 69 79 71 77 71 74 71 74 72 73 70 71 76
 70 70 71 71 73 72 74 72 76 72 67 73 79 76 72 70 69 72 76 69 70 72 74 72
 73 77]
#Gen 3:
[79 72 79 74 72 77 79 76 74 77 77 82 79 76 76 80 75 77 80 76 74 74 73 75
 73 77 79 76 76 77 77 74 75 74 80 79 82 77 79 79 74 77 80 76 76 74 79 74
 71 76]
#Gen 4:
[78 76 76 79 88 81 76 81 81 80 80 79 82 82 82 79 81 83 79 79 80 83 80 79
 76 81 80 82 78 77 77 79 80 76 75 83 80 81 82 81 79 83 77 79 82 79 88 77
 79 82]
#Gen 5:
[83 84 81 83 83 87 88 82 80 81 82 84 83 84 

In [20]:
np.random.seed(20520864)
num_variables = 128
num_individuals = 2
num_generations = 50
better_algorithm = POPOP(initialization=initialize_population, variation=better_variation, evaluation=onemax)
better_algorithm(num_individuals, num_variables, num_generations)
# Converge at 1th generation

#First Population:
[[1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1 1 1 0 0 0 0 1
  1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1
  1 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1
  0 0 0 1 1 0 0 0 1 1 1 1 0 1 0 1 0 1 1 0]
 [0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1 1 1 0 0 0 1 1 1 1 0
  0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0
  0 1 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0
  1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 1]]
#Gen 0:
[67 61]
#Gen 1:
[128 128]
Population converged
#Best Individual:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
### Nhận xét ###
# Em dò tay ra kích thước quần thể nhỏ nhất đảm bảo hội tụ ra lời giải tối ưu trong vòng 50 thế hệ.
# Nhưng cả kích thước quần thể và số thế hệ đều lớn.
# Kết luận: Phép biến đổi của genetic algorithm quá kém hiệu quả